In [ ]:
!pip install torch transformers sentence_transformers langchain langchain_together dspy
!pip install -U langchain-community

In [ ]:
import os
together_key = os.getenv("TOGETHER_API_KEY")

In [ ]:
# together_ai/google/gemma-2-9b-it together_ai/meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo
import dspy
lm = dspy.LM('together_ai/meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo', api_key=together_key, cache = False, max_tokens=1500)
dspy.configure(lm=lm)

In [ ]:
## codigo dos direitos do autor
import json
from datetime import datetime
with open('codigo_estrada.json', 'r') as file:
    context_data = json.load(file)

with open('qa_estrada.json', 'r') as file:
    qa_json = json.load(file)

## LLM Metric

In [ ]:
metricLM = dspy.LM('together_ai/google/gemma-2-9b-it', cache = False, api_key=together_key)


In [ ]:
class EvaluateAnswer(dspy.Signature):
    """Avalia a qualidade da resposta do sistema a uma certa questão. Tendo em conta os seguintes fatores:
        - A resposta é suportada pelo contexto?
        - A resposta responde corretamente à questão?
        - A resposta está totalmente com base no contexto?
        - Por favor avalia quão bem a resposta dada responde à questão baseado no contexto fornecido."""

    context = dspy.InputField(desc="O contexto necessário para responder à questão")
    question = dspy.InputField(desc="A pergunta perguntada ao sistema")
    model_answer = dspy.InputField(desc="A resposta do sistema que tens de avaliar")
    rating = dspy.OutputField(desc="Resultado entre 1 e 5. Apenas responde o número como um `int`, nada mais!")

def rag_judge(gold, pred, trace=None):
  predicted_answer = pred.answer
  question = gold.question
  context = gold.context
  init_time = datetime.now()

  with dspy.context(lm=metricLM):
      faithful = dspy.ChainOfThought(EvaluateAnswer)(context=context, question=question, model_answer=predicted_answer)

  print(f"Faithful: {faithful.rating}")

  final_time = datetime.now()
  print(f"Time passed: {final_time-init_time}\n")
  return float(faithful.rating)




In [ ]:
correct_answers= 0
for qa in qa_json:
    eval_score = rag_judge(context=context_data, question=qa['question'], predicted_answer=qa['answer'])
    correct_score = qa['score']
    print(f"correct: {correct_score}, predict: {eval_score}\n")

    if correct_score == round(eval_score) :
        correct_answers += 1

total_questions = len(qa_json)
accuracy_percentage = (correct_answers / total_questions) * 100

In [ ]:
accuracy_percentage

**Optimizer**

In [ ]:
from dspy.teleprompt import BootstrapFewShot

optimizer = BootstrapFewShot(
    metric=eval_score,
    max_bootstrapped_demos=1,
    max_labeled_demos=3,
    max_rounds=10,
)

In [ ]:
import pandas as pd
qa_df = pd.DataFrame(qa_json)

In [ ]:
def get_dspy_df(data):
    train_df = []
    for _, row in data.iterrows():
        train_df.append(dspy.Example(context=row['context'], question=row['question'], answer=row['answer'], score=row['score']).with_inputs('question', 'context', 'answer'))
    return train_df

In [ ]:
qa_train = get_dspy_df(qa_df)

In [ ]:
cot_compiled = optimizer.compile(RAGMetricProgram(), trainset=qa_train)

In [ ]:
cot_compiled(context=context_data, question=test_q2, answer= test_pred)

In [ ]:
cot_compiled.save("compiled_cdadc.json")

## RAG

CoT with Hint

In [ ]:
from enum import Enum
import re
class LegalCode(Enum):
    TRABALHO = "Código do Trabalho e Processo do Trabalho"
    PREDIAL = "NRAU e Código do Registo Predial"
    CDADC = "Código dos Direitos de Autor"
    CIRS = "Código IRS"
    CIMI = "Código do IMI e Código do IMT"
    CN = "Código do Notariado"
    ESTRADA = "Código da Estrada"
    CIRE = "Codigo da Insolvencia e da Recuperacao de Empresas"
    CCP = "Codigo Contratos Publicos"

class SpecialistPrompts:
    def __init__(self):
      self.DICIONARIO = {
            LegalCode.ESTRADA: {
                "buzina": "avisador",
                "acidente": "sinistro",
                "sinal": "pisca",
                "velocípedes": "bicicletas",
                "transito": "fila",
                "veículo de transporte coletivo de passageiros": "autocarros",
                "veículo de transporte coletivo de carga": "camião"
            }
        }
      self.INTRODUCTIONS = {
            LegalCode.TRABALHO: "És um especialista em Direito Laboral português. Tens profundo conhecimento do Código do Trabalho e do Processo do Trabalho, bem como da legislação relativa ao emprego em Portugal.",
            LegalCode.PREDIAL: "És um especialista em legislação de arrendamento e registo predial em Portugal. Dominas o Novo Regime do Arrendamento Urbano (NRAU) e o Código do Registo Predial.",
            LegalCode.CCP: "És um especialista em contratação pública em Portugal. Tens conhecimento aprofundado do Código dos Contratos Públicos e da legislação aplicável aos contratos celebrados por entidades públicas.",
            LegalCode.CDADC: "És um especialista em legislação de propriedade intelectual em Portugal. Tens profundo conhecimento do Código dos Direitos de Autor e Direitos Conexos, com foco na proteção e gestão de obras intelectuais.",
            LegalCode.CIRE: "És um especialista em insolvência e recuperação de empresas em Portugal. Dominas o Código de Insolvência e Recuperação de Empresa e toda a legislação aplicável aos processos de recuperação e falência de empresas.",
            LegalCode.CIRS: "És um especialista sobre o Codigo do IRS - imposto sobre o Rendimsento das Pessoas Singulares. Tens profundo conhecimento do Código do IRS, incluindo as normas e obrigações relacionadas com esse imposto em Portugal.",
            LegalCode.CIMI: "És um especialista em impostos sobre o património imobiliário em Portugal. Dominas o Código do IMI (Imposto Municipal sobre Imóveis) e o Código do IMT (Imposto Municipal sobre as Transmissões Onerosas de Imóveis).",
            LegalCode.CN: "És um especialista em direito notarial em Portugal. Tens profundo conhecimento do Código do Notariado e das normas relativas à autenticação e formalização de atos e documentos legais.",
           # LegalCode.ESTRADA: "És um especialista em legislação rodoviária em Portugal. Dominas o Código da Estrada e toda a regulamentação relacionada com a segurança, regras de trânsito e legislação para condutores e veículos."
            LegalCode.ESTRADA: "És um especialista em legislação rodoviária em Portugal. Aqui tens um dicionário de sinónimos com termos utilizados no Código da Estrada "
        }
    def _get_legal_code(self,input_string: str) -> LegalCode:
      patterns = {
          LegalCode.TRABALHO: r'\b(trabalho|processo do trabalho)\b',
          LegalCode.PREDIAL: r'\b(nrau|registo predial|predial)\b',
          LegalCode.CDADC: r'\b(direitos de autor|cdadc)\b',
          LegalCode.CIRS: r'\b(cirs|código irs)\b',
          LegalCode.CIMI: r'\b(cimi|código do imi|imt)\b',
          LegalCode.CN: r'\b(código do notariado|notariado|cn)\b',
          LegalCode.ESTRADA: r'\b(código da estrada|estrada)\b',
          LegalCode.CIRE: r'\b(cire|insolvência|recuperação de empresas)\b',
          LegalCode.CCP: r'\b(ccp|código contratos públicos)\b',
      }

      for code, pattern in patterns.items():
          if re.search(pattern, input_string, re.IGNORECASE):
              return code
      return None


In [ ]:
import pydantic
from typing import List

class References(pydantic.BaseModel):
    article_title : str
    url : str
    date : str

class StructuredAnswer(pydantic.BaseModel):
    answer: str
    references: List[References]

In [ ]:
class GenerateAnswer(dspy.Signature):
    """Sendo tu um especialista da legislação portuguesa, responde à questão baseado EXCLUSIVAMENTE no contexto fornecido. Foca-te em fornecer informacões corretas e fundamenta com o contexto"""
    context = dspy.InputField(desc="Informação importante para responder à questão")
    question = dspy.InputField()
    answer : StructuredAnswer = dspy.OutputField(desc="Uma resposta detalhada e a lista de referencias utilizadas para responder à questão (título do artigo, url e data)")

In [ ]:
specialist = SpecialistPrompts()

In [ ]:
class RAG(dspy.Module):
    def __init__(self):
        super().__init__()
        # self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.generate_answer = dspy.ChainOfThoughtWithHint(GenerateAnswer)
    def forward(self, question, context, hint):
        init_time = datetime.now()
        pred = self.generate_answer(context=context, question=question, hint= hint).answer
        res = dspy.Prediction(context=context, answer=pred, question=question)
        final_time = datetime.now()

        print(f"Time passed RAG: {final_time-init_time}\n")
        return res

testing without optimizer

In [ ]:
import numpy as np
from langchain.embeddings import SentenceTransformerEmbeddings

embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))



def text_evaluation(correct_answer, model_answer, trace=None):
  correct_answer_embeddings = embedding.embed_query(correct_answer.answer.answer)#.gold_answer)
  model_answer_embedding = embedding.embed_query(model_answer)

  similarity = cosine_similarity(model_answer_embedding, correct_answer_embeddings)
  return similarity




<ipython-input-12-ab7336128e2e>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warn

In [ ]:
def get_dspy_rag(data, contexto):
    train_df = []
    for row in data['train']:
      train_df.append(dspy.Example(context=contexto, question=row['question'], code=row['code'], gold_answer=row['answer']).with_inputs('question', 'context', 'code'))
    return train_df

In [ ]:
qa_train = get_dspy_rag(qa_json, context_data)

In [ ]:
# from dspy.teleprompt import BootstrapFewShot

# optimizer = BootstrapFewShot(
#     metric=rag_judge,
#     max_bootstrapped_demos=1,
#     max_labeled_demos=4,
#     max_rounds=10,
# )

In [ ]:
rag_compiled = optimizer.compile(RAG(), trainset=qa_train)

In [ ]:
rag_compiled.save("estrada_compiled_hint.json")

---------------------------------------------------------------------------------------

In [ ]:
loaded_rag = RAG()
# loaded_rag.load("estrada_compiled_ragjudge.json")

cos_sim_final = 0
for qa in qa_json['test']:
  legal_code = specialist._get_legal_code(qa['code'])
  introduction = specialist.INTRODUCTIONS.get(legal_code)
  dictionary = specialist.DICIONARIO.get(legal_code)
  hint = introduction + str(dictionary)
  rag_answer = loaded_rag(context=context_data, question=qa['question'], hint = hint)
  cos_sim = text_evaluation(rag_answer, qa['answer'])
  print(f"for the question {qa['question']} LLAMA answers\n ##{rag_answer.answer}## COS: {cos_sim}\n")
  cos_sim_final+=cos_sim



Time passed RAG: 0:00:03.183587

for the question Quando é que o trânsito pode passar pela esquerda de postes ou ilhéus? LLAMA answers
 ##answer='O trânsito pode passar pela esquerda de postes ou ilhéus quando se encontrarem no eixo da faixa de rodagem de que procedem os veículos, ou em vias de sentido único ou em faixas de rodagem afetas a um só sentido.' references=[References(article_title='Placas, postes, ilhéus e dispositivos semelhantes', url='https://www.segurancarodoviaria.pt/codigo-da-estrada/titulo-ii-do-transito-de-veiculos-e-animais/capitulo-i-disposic%C3%B5es-comuns/sec%C3%A7%C3%A3o-i-regras-gerais/artigo-16o-placas-postes-ilheus-e-dispositivos-semelhantes/?returnUrl=%2fcodigo-da-estrada%2f%3fshow%3d2110%23a-seccao-2284', date='1985-09-22')]## COS: 0.6164475680237272

Time passed RAG: 0:00:02.691888

for the question A minha filha, de 6 anos, pode andar de bicicleta no passeio? LLAMA answers
 ##answer='Sim, a sua filha pode andar de bicicleta no passeio, desde que não ponh

In [ ]:
cos_sim_final

4.579515037595461

In [ ]:
lm.inspect_history(n=2)





[2024-11-18T09:20:45.145258]

System message:

Your input fields are:
1. `context` (str): Informação importante para responder à questão
2. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (StructuredAnswer): Uma resposta detalhada e a lista de referencias utilizadas para responder à questão (título do artigo, url e data)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}        # note: the value you produce must be pareseable according to the following JSON schema: {"type": "object", "$defs": {"References": {"type": "object", "properties": {"article_title": {"type": "string", "title": "Article Title"}, "date": {"type": "string", "title": "Date"}, "url": {"type": "string", "title": "Url"}}, "required": ["article_title", "url", "date"], "title": "References"}}, "properties": {"answer":